# Importing the Libraries

In [ ]:
import numpy as np
np.set_printoptions(threshold=np.inf)
import pandas as pd
import matplotlib.pyplot as plt

# Importing Dataset

In [ ]:
titanic_train = pd.read_csv('train.csv')
titanic_test = pd.read_csv('test.csv')

# Analysing Data

In [ ]:
titanic_train.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [ ]:
titanic_train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
titanic_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


# Finding Missing Values

In [ ]:
titanic_train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
titanic_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# Dropping redundant features

In [ ]:
titanic_train = titanic_train.drop(columns=['Name','Cabin','Ticket'])

In [ ]:
titanic_test = titanic_test.drop(columns=['Name','Cabin','Ticket'])

# Converting Dataframes to multidimensional arrays

In [ ]:
# Titanic train set is stored in an array
X_1 = titanic_train.iloc[:,0:1].values
X_2 = titanic_train.iloc[:,2:].values
X_train = np.concatenate((X_1,X_2), axis=1)

# Survied column stored in the form of vector 
y_train = titanic_train.iloc[:,1].values 

In [ ]:
# Titanic test set stored in an array
X_test = titanic_test.iloc[:,:].values

# Handling missing data in training and test set

In [ ]:
from sklearn.impute import SimpleImputer

# Adding 'median' in missing age values for training set
imputer_age = SimpleImputer(missing_values = np.nan, strategy = 'median')
imputer_age.fit(X_train[:,3:4])
X_train[:,3:4] = imputer_age.transform(X_train[:,3:4])

In [ ]:
# Adding 'most_frequent' in missing embarked values for training set
imputer_embarked = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
imputer_embarked.fit(X_train[:,-1:])
X_train[:,-1:] = imputer_embarked.transform(X_train[:,-1:])

In [ ]:
# Adding 'median' in missing age values for test set
imputer_age.fit(X_test[:,3:4])
X_test[:,3:4] = imputer_age.transform(X_test[:,3:4])

In [ ]:
# Adding 'mean' in missing fare values for test set
imputer_fare = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputer_fare.fit(X_test[:,-2:-1])
X_test[:,-2:-1] = imputer_fare.transform(X_test[:,-2:-1])

# Encoding Categorical Data

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoding gender in training set
ct_gender = ColumnTransformer(transformers = [('encoder', OneHotEncoder(),[2])],remainder = 'passthrough')
X_train = np.array(ct_gender.fit_transform(X_train))

In [ ]:
# OneHotEncoding Embarked in training set
ct_embarked = ColumnTransformer(transformers = [('encoder', OneHotEncoder(),[-1])],remainder = 'passthrough')
X_train = np.array(ct_embarked.fit_transform(X_train))

In [ ]:
# OneHotEncoding gender in test set
X_test = np.array(ct_gender.fit_transform(X_test))

In [ ]:
# OneHotEncoding Embarked in test set
X_test = np.array(ct_embarked.fit_transform(X_test))

# Feature Scaling

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# Bringing features on same scale for both set
X_train[:,5:6] = sc.fit_transform(X_train[:,5:6])
X_train[:,7:] = sc.fit_transform(X_train[:,7:])
X_test[:,5:6] = sc.fit_transform(X_test[:,5:6])
X_test[:,7:] = sc.fit_transform(X_test[:,7:])


In [ ]:
# Converting the arrays from object to float64
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')

# Training Random Forest on training set

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
# KNN
# from sklearn.neighbors import KNeighborsClassifier
# classifier = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p=2)
# classifier.fit(X_train,y_train)

# Logistic Regression
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression(random_state = 0)
# classifier.fit(X_train,y_train)

# Naive Bayes
# from sklearn.naive_bayes import GaussianNB
# classifier = GaussianNB()
# classifier.fit(X_train, y_train)

# Kernel SVM
# from sklearn.svm import SVC
# classifier = SVC(kernel = 'rbf', random_state = 0)
# classifier.fit(X_train, y_train)

# Validating the Test Set for Random Forest

In [ ]:
y_test = classifier.predict(X_test)

# Accuracy

In [ ]:
from sklearn.metrics import accuracy_score
classifier.score(X_train, y_train)
classifier = round(classifier.score(X_train, y_train) * 100, 2)
classifier

97.76

# Creating CSV File

In [ ]:
validator = pd.DataFrame({
        "PassengerId": titanic_test["PassengerId"],
        "Survived": y_test
    })

In [ ]:
validator.to_csv('submission_randomForest.csv', index=False)